### Import packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
import math

### Load data

In [178]:
path = '/Users/jooyong/github_locals/Soccer_Player_Recommendation_Service/data/player_stats(from_whoscored)/Premier_league_player_stats/2017-2018/'
file_list = os.listdir(path)

Premier_matches = pd.DataFrame()
for file in file_list:

    filepath = path + file
    Premier_matches = pd.concat([Premier_matches, pd.read_json(filepath, lines=True)], axis=0)

### High level analysis

In [179]:
Premier_matches.head()

,date,home_name,away_name,home_lineup,away_lineup
0,"Jtat, 16-Okt-17",Leicester,West Bromwich Albion,"[{'Name': 'Kasper Schmeichel', 'Position': 'GK...","[{'Name': 'Boaz Myhill', 'Position': 'GK', 'Sh..."
1,"Ijm, 20-Okt-17",West Ham,Brighton,"[{'Name': 'Joe Hart', 'Position': 'GK', 'Shots...","[{'Name': 'Mathew Ryan', 'Position': 'GK', 'Sh..."
2,"Jmos, 21-Okt-17",Chelsea,Watford,"[{'Name': 'Thibaut Courtois', 'Position': 'GK'...","[{'Name': 'Heurelho Gomes', 'Position': 'GK', ..."
3,"Jmos, 21-Okt-17",Stoke,Bournemouth,"[{'Name': 'Jack Butland', 'Position': 'GK', 'S...","[{'Name': 'Asmir Begovic', 'Position': 'GK', '..."
4,"Jmos, 21-Okt-17",Swansea,Leicester,"[{'Name': 'Lukasz Fabianski', 'Position': 'GK'...","[{'Name': 'Kasper Schmeichel', 'Position': 'GK..."


In [180]:
Premier_matches.isnull().sum()

date           0
home_name      0
away_name      0
home_lineup    0
away_lineup    0
dtype: int64

In [181]:
Premier_matches.describe()

,date,home_name,away_name,home_lineup,away_lineup
count,401,401,401,401,401
unique,105,20,20,377,368
top,"Jmos, 12-Ago-17",Watford,Tottenham,"[{'Name': 'Heurelho Gomes', 'Position': 'GK', ...",[]
freq,21,22,21,3,10


### Player stats analysis

In [182]:
line_up_stats = pd.concat([Premier_matches[['date', 'home_lineup']].rename(columns={'home_lineup':'line_up'}), Premier_matches[['date', 'away_lineup']].rename(columns={'away_lineup':'line_up'})], axis=0).reset_index(drop=True)

In [183]:
line_up_stats.head()

,date,line_up
0,"Jtat, 16-Okt-17","[{'Name': 'Kasper Schmeichel', 'Position': 'GK..."
1,"Ijm, 20-Okt-17","[{'Name': 'Joe Hart', 'Position': 'GK', 'Shots..."
2,"Jmos, 21-Okt-17","[{'Name': 'Thibaut Courtois', 'Position': 'GK'..."
3,"Jmos, 21-Okt-17","[{'Name': 'Jack Butland', 'Position': 'GK', 'S..."
4,"Jmos, 21-Okt-17","[{'Name': 'Lukasz Fabianski', 'Position': 'GK'..."


Because goal keeper and other field players have different stat features, two dataframe:one for goalkeeper and one for field player should be made.

In [184]:
gk_player_stat_list = []

field_player_stat_list = []

for i in line_up_stats.index:

    date = line_up_stats.iloc[i]['date']

    for player in line_up_stats.iloc[i]['line_up']:

        if player['Position']=='GK':

            player['date']=date

            gk_player_stat_list.append(player)

        else:

            player['date']=date
            
            field_player_stat_list.append(player)



In [185]:
field_player_stat_df = pd.DataFrame(field_player_stat_list)

In [186]:
field_player_stat_df['Position'].unique()

array(['DR', 'DC', 'DL', 'MR', 'MC', 'ML', 'AMC', 'FW', 'Sub', 'DMC',
       'AMR', 'AML', 'DMR', 'DML', 'FWR', 'FWL'], dtype=object)

In [187]:
field_player_stat_df[field_player_stat_df['Position']=='FWL']['Name'].unique()

array(['Eden Hazard', 'Jesé', 'Jordan Ayew', 'Leroy Sané',
       'Marko Arnautovic', 'Gylfi Sigurdsson', 'Raheem Sterling',
       'Roberto Firmino', 'Eric Choupo-Moting', 'Ramadan Sobhi',
       'Sadio Mané', 'Matt Phillips', 'Mame Diouf', 'Richarlison',
       'Philippe Coutinho', 'Roberto Pereyra', 'José Izquierdo',
       'Marc Pugh', 'Pierre-Emerick Aubameyang', 'Wayne Routledge',
       'Gabriel Jesus', 'Jay Rodríguez', 'Yannick Bolasie', 'Danny Ings',
       'Nathan Redmond', 'Alexis Sánchez', 'Pedro', 'Wilfried Zaha',
       'Danny Welbeck', 'Nathan Dyer', 'James McClean', 'Marc Albrighton',
       'Wayne Rooney', 'Chicharito', 'Alexander Sørloth', 'Nacer Chadli',
       'Henrikh Mkhitaryan', 'Jeffrey Schlupp', 'Anthony Martial',
       'Joshua King', 'Ryan Fraser', 'Ademola Lookman', 'Manuel Lanzini',
       'Jürgen Locadia', 'Dominic Calvert-Lewin', 'Alex Iwobi',
       'Junior Stanislas', 'Marcus Rashford', 'Lee Chung-Yong'],
      dtype=object)

In [188]:
# position is wrong...position information of player should be saved in other dataset..
Nani_stats_2010_2011 = field_player_stat_df[field_player_stat_df['Name']=='Sadio Mané']

In [189]:
Nani_stats_2010_2011[~Nani_stats_2010_2011.duplicated()]

,Name,Position,Shots,ShotsOT,KeyPasses,PA%,AerialsWon,Touches,Rating,Dribbles,...,BlockedShots,Fouls,Passes,Crosses,AccCrosses,LB,AccLB,ThB,AccThB,date
381,Sadio Mané,FWR,5,4,1,80.8,1,77,7.66,2,...,0,1,52,3,0,2,1,0,0,"Jmos, 19-Ago-17"
587,Sadio Mané,FWL,0,0,3,80,2,57,8.13,1,...,0,0,40,1,0,2,2,1,1,"Jmos, 17-Mac-18"
1131,Sadio Mané,FWL,3,2,0,90.6,0,45,7.94,1,...,0,0,32,2,0,0,0,0,0,"Jpl, 27-Ago-17"
2168,Sadio Mané,FWL,2,1,5,80.4,0,79,7.34,5,...,0,0,51,3,0,2,2,0,0,"Jpl, 10-Des-17"
2837,Sadio Mané,Sub,0,0,0,0,0,0,-,0,...,0,0,0,0,0,0,0,0,0,"Jnne, 26-Des-17"
2933,Sadio Mané,FWL,1,0,3,88.9,0,52,6.85,0,...,0,0,45,0,0,1,0,0,0,"Jmos, 30-Des-17"
4021,Sadio Mané,FWL,2,0,3,87.5,1,54,7.07,2,...,0,3,32,3,1,0,0,0,0,"Jmos, 18-Nov-17"
4308,Sadio Mané,ML,3,1,5,65.5,2,39,7.22,1,...,0,1,29,0,0,3,1,0,0,"Jpl, 13-Mei-18"
4611,Sadio Mané,MR,2,0,0,62.1,2,50,7.05,3,...,0,2,29,1,0,1,1,0,0,"Jtan, 13-Des-17"
4973,Sadio Mané,FWL,3,1,1,77.8,0,47,8.28,3,...,0,2,27,0,0,1,1,1,0,"Jpl, 14-Jan-18"
